In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from tqdm import tqdm

main_directory = '/content/drive/MyDrive/Data Science CapstoneProject/Bicing_Capstone/1.DATA'

df2021_1 = pd.read_parquet(os.path.join(main_directory, 'preprocessed2021_1to6.parquet'))
df2021_2 = pd.read_parquet(os.path.join(main_directory, 'preprocessed2021_7to12.parquet'))
df2022_1 = pd.read_parquet(os.path.join(main_directory, 'preprocessed2022_1to6.parquet'))
df2022_2 = pd.read_parquet(os.path.join(main_directory, 'preprocessed2022_7to12.parquet'))
df2023_1 = pd.read_parquet(os.path.join(main_directory, 'preprocessed2023_1to6.parquet'))
df2023_2 = pd.read_parquet(os.path.join(main_directory, 'preprocessed2023_7to12.parquet'))

In [ ]:
print(len(df2021_1))
print(len(df2021_2))
print(len(df2022_1))
print(len(df2022_2))
print(len(df2023_1))
print(len(df2023_2))

1812440
2215377
1818296
2228821
2048872
2058799


In [ ]:
dfs = [df2021_1, df2021_2, df2022_1, df2022_2, df2023_1, df2023_2]
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df.groupby(['year','month']).count()

station_id  num_docks_available     day    hour
year month                                                 
2021 1          376056               376056  376056  376056
     2          334685               334685  334685  334685
     3            1526                 1526    1526    1526
     4          365456               365456  365456  365456
     5          376738               376738  376738  376738
     6          357979               357979  357979  357979
     7          370452               370452  370452  370452
     8          375673               375673  375673  375673
     9          364796               364796  364796  364796
     10         375956               375956  375956  375956
     11         360305               360305  360305  360305
     12         368195               368195  368195  368195
2022 1          367915               367915  367915  367915
     2          340894               340894  340894  340894
     3            1527                 1527    1527    1527
     4          365594               365594  365594  365594
     5          377691               377691  377691  377691
     6          364675               364675  364675  364675
     7          376818               376818  376818  376818
     8          375118               375118  375118  375118
     9          363496               363496  363496  363496
     10         375503               375503  375503  375503
     11         362036               362036  362036  362036
     12         375850               375850  375850  375850
2023 1          339090               339090  339090  339090
     2          338661               338661  338661  338661
     3          359097               359097  359097  359097
     4          302533               302533  302533  302533
     5          354989               354989  354989  354989
     6          354502               354502  354502  354502
     7          378941               378941  378941  378941
     8          259211               259211  259211  259211
     9          312739               312739  312739  312739
     10         374219               374219  374219  374219
     11         356779               356779  356779  356779
     12         376910               376910  376910  376910

In [ ]:
path_MD = '/content/drive/MyDrive/Data Science CapstoneProject/Bicing_Capstone/INFO ESTACIONS BICING/Informacio_Estacions_Bicing.csv'

In [ ]:
def create_context(df, path_MD):
    df['station_id'] = df['station_id'].astype('int')
    df_info = pd.read_csv(path_MD)
    df = df.merge(df_info[['station_id', 'capacity','lat','lon','altitude','address']], on='station_id', how='inner')

    df['percentage_docks_available'] = round((df['num_docks_available'] / df['capacity']).clip(upper=1.0),2)

    df = df.sort_values(by=['station_id','year','month', 'day', 'hour'])

    # Crear las columnas de contexto
    for i in range(1, 5):
        df[f'ctx_{i}'] = df.groupby('station_id')['percentage_docks_available'].shift(i)

    return df

In [ ]:
df = create_context(df, path_MD)

In [ ]:
df

,station_id,num_docks_available,month,day,hour,year,capacity,lat,lon,altitude,address,percentage_docks_available,ctx_1,ctx_2,ctx_3,ctx_4
0,1,5.500000,1,1,0,2021,45,41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",0.12,NaN,NaN,NaN,NaN
1,1,5.000000,1,1,1,2021,45,41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",0.11,0.12,NaN,NaN,NaN
2,1,5.000000,1,1,2,2021,45,41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",0.11,0.11,0.12,NaN,NaN
3,1,5.000000,1,1,3,2021,45,41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",0.11,0.11,0.11,0.12,NaN
4,1,5.000000,1,1,4,2021,45,41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",0.11,0.11,0.11,0.11,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12073737,519,18.416667,12,31,19,2023,24,41.424655,2.166289,110.0,"C/ PEDRELL, 52",0.77,0.87,0.85,0.83,0.89
12073738,519,17.166667,12,31,20,2023,24,41.424655,2.166289,110.0,"C/ PEDRELL, 52",0.72,0.77,0.87,0.85,0.83
12073739,519,16.416667,12,31,21,2023,24,41.424655,2.166289,110.0,"C/ PEDRELL, 52",0.68,0.72,0.77,0.87,0.85
12073740,519,16.000000,12,31,22,2023,24,41.424655,2.166289,110.0,"C/ PEDRELL, 52",0.67,0.68,0.72,0.77,0.87


In [ ]:
df.isnull().sum()

station_id                       0
num_docks_available              0
month                            0
day                              0
hour                             0
year                             0
capacity                         0
lat                              0
lon                              0
altitude                         0
address                          0
percentage_docks_available       0
ctx_1                          506
ctx_2                         1012
ctx_3                         1518
ctx_4                         2024
dtype: int64

In [ ]:
# Save the final DataFrame as a pickle file
parquet_path = '/content/drive/MyDrive/Data Science CapstoneProject/Bicing_Capstone/3. EDA/data_ready2.parquet'
df.to_parquet(parquet_path)